In [1]:
#IMPORTAMOS LAS LIBRERIAS QUE VAMOS A USAR
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import IncrementalPCA
from sklearn.model_selection import train_test_split

In [2]:
#abro el dataset
df_ML1= pd.read_csv('df_ML1.csv')

In [3]:
df_ML1

,Unnamed: 0,genres,app_name,price_x,item_id,developer,user_id,playtime_forever,recommend,sentiment_analysis,genres_encoded
0,0,Action,Half-Life,9.99,70,Valve,DingarooEF,108.0,True,2,0
1,252,"Action, Indie, RPG",The Ship: Murder Party,9.99,2400,Outerlight Ltd.,76561198151870370,1.0,True,0,107
2,580,"Indie, Strategy",DEFCON,9.99,1520,Introversion Software,76561198076145661,158.0,True,2,270
3,679,"Action, Indie, RPG",The Ship: Single Player,9.99,2420,Outerlight Ltd.,UberDuckLord,15.0,True,2,107
4,712,"Indie, Simulation",Garry's Mod,9.99,4000,Facepunch Studios,euzi,412.0,True,1,263
...,...,...,...,...,...,...,...,...,...,...,...
1855,185998,Action,Deathmatch Classic,4.99,40,Valve,76561198038840599,4.0,True,2,0
1856,186013,Action,Half-Life: Blue Shift,4.99,130,Gearbox Software,efey123,313.0,True,2,0
1857,186077,Action,Day of Defeat,4.99,30,Valve,evcentric,7.0,True,2,0
1858,186133,Action,Unreal Tournament 2004: Editor's Choice Edition,14.99,13230,"Epic Games, Inc.",PimpIt,1668.0,True,2,0


In [4]:
#se eliminan columnas inaportantes
df_ML1=df_ML1.drop(columns=['Unnamed: 0','developer', 'playtime_forever'])

In [5]:
#creo el modelo de machine learning (simulitud de cosenos)
#vectorizamos los datos
textos = df_ML1['genres']
vectorizer = TfidfVectorizer()
matriz_tfidf = vectorizer.fit_transform(textos)

In [6]:
#Reducimos dimencionalidad con PCA
ipca = IncrementalPCA(n_components=20)
matriz_ipca = ipca.fit_transform(matriz_tfidf)

In [7]:
#Hacemos el cálculo de similitud del coseno después de PCA
similitud_pca = cosine_similarity(matriz_ipca)
similitud = similitud_pca

In [9]:
'''Como queremos recomendar juegos, hacemos un filtro de la columna sentiment_analysis==2 y recommend==True ya que estos son los positivos, y necesitamos usar menos memoria'''
df_ML1=df_ML1[df_ML1['sentiment_analysis']==2]
df_ML1=df_ML1[df_ML1['recommend']== True]

In [ ]:
'''continua sin funcionar, no tengo los recursos de memoria Ram para aplicar el modelo de similaridad del coseno'''

In [35]:
#Uso el modelo de k-Nearest Neighbors 
#Importo la libreria picke (formato .pkl archivo de formato binario)

import pickle

In [36]:
#CREO UNA INSTANCIA DEL CODIFICADOR
label_encoder = LabelEncoder()

In [37]:
# Cargar los datos
df_ml1 = pd.read_csv("df_ML1.csv")

In [39]:
#se eliminan columnas inaportantes
df_ml1=df_ml1.drop(columns=['Unnamed: 0'])

In [40]:
# se crea una nueva columna llamada genres_encoded, que tiene los generos codificados como int.
df_ml1["genres_encoded"] = label_encoder.fit_transform(df_ml1["genres"])

In [41]:
# se crea un diccionario para los títulos asociados a cada item_id
titles_by_item_id = {}
for i in range(len(df_ml1)):
    titles_by_item_id[df_ml1.loc[i, "item_id"]] = df_ml1.loc[i, "app_name"]

In [42]:
# creo el modelo
k = 5
model = KNeighborsClassifier(n_neighbors=k)

In [43]:
# Entreno el modelo
model.fit(df_ml1[['genres_encoded']], df_ml1['app_name'])

KNeighborsClassifier()

In [64]:
df_ML1.sample()

,genres,app_name,price_x,item_id,user_id,recommend,sentiment_analysis,genres_encoded
275,"Indie, Strategy",Unity of Command: Stalingrad Campaign,19.99,218090,76561198036297711,True,2,270


In [65]:
# Guardo el modelo1
with open('modelo1.pkl', 'wb') as f:
    pickle.dump(model, f)
# Guardo el diccionario
with open('titles_by_item_id.pkl', 'wb') as f:
    pickle.dump(titles_by_item_id, f)
def get_recommendations(item_id: int):

    # Cargar el modelo1 
    with open("modelo1.pkl", "rb") as f:
        model = pickle.load(f)

    # Cargar el diccionario de títulos
    with open("titles_by_item_id.pkl", "rb") as f:
        titles_by_item_id = pickle.load(f)

    # Buscar el género codificado del juego proporcionado por el usuario
    input_game = df_ml1[df_ml1["item_id"] == item_id]["genres_encoded"].values[0]

    # Encontrar los juegos más similares
    _, indices = model.kneighbors([[input_game]])

    # Obtener los títulos de los juegos similares
    similar_games = [titles_by_item_id[df_ml1.loc[i, "item_id"]] for i in indices[0]]

    # Devolver un diccionario de los títulos
    return {"similar_games": similar_games}
get_recommendations(218090)

c:\Users\Elena\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


{'similar_games': ['DEFCON',
  'StarDrive',
  'Defense Grid: The Awakening',
  'Frozen Synapse',
  'Cubemen']}